# `requests` 통해 웹 정보를 받아 크롤링

In [1]:
import requests

### 테스트

In [6]:
## https://comic.naver.com/webtoon 에 접속했을 때, 요일별 현재 연재 중인 전체 웹툰이 담긴 api
url = "https://comic.naver.com/api/webtoon/titlelist/weekday?order=user"

head  = {'User-Agent':
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'}

r = requests.get(url, headers=head)
# r.text

In [7]:
## 하나의 웹툰 찾기 
r.json()['titleListMap']['TUESDAY'][0]

{'titleId': 783053,
 'titleName': '김부장',
 'author': '박태준 만화회사 / 정종택',
 'thumbnailUrl': 'https://image-comic.pstatic.net/webtoon/783053/thumbnail/thumbnail_IMAG21_d7732f14-26da-4e35-8762-660cc87b53f1.jpg',
 'up': False,
 'rest': False,
 'bm': False,
 'adult': False,
 'starScore': 9.47574,
 'viewCount': 0,
 'openToday': False,
 'potenUp': False,
 'bestChallengeLevelUp': False,
 'finish': False,
 'new': False}

In [8]:
### 화요일 웹툰 titleId 모으기 테스트

a = []
for i in range(r.json()['titleListMap']['TUESDAY'].__len__()):
    a.append(r.json()['titleListMap']['TUESDAY'][i]['titleId'])
a[:6]

[783053, 784248, 796152, 790713, 789979, 730656]

In [9]:
r.json()['titleListMap'].keys()

dict_keys(['MONDAY', 'SUNDAY', 'TUESDAY', 'SATURDAY', 'THURSDAY', 'FRIDAY', 'WEDNESDAY'])

## 요일 전체 `titleId` 가져오기

In [10]:
import requests
from tqdm import tqdm

In [11]:
## https://comic.naver.com/webtoon 에 접속했을 때, 요일별 현재 연재 중인 전체 웹툰이 담긴 api
url = "https://comic.naver.com/api/webtoon/titlelist/weekday?order=user"

head  = {'User-Agent':
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'}

r = requests.get(url, headers=head)

In [12]:
week_day = list(r.json()['titleListMap'].keys()) # 요일 담은 리스트

titleId_list =[]
for i in tqdm(week_day):
    for j in range(r.json()['titleListMap'][i].__len__()):
        titleId_list.append(r.json()['titleListMap'][i][j]['titleId'])


100%|██████████| 7/7 [00:01<00:00,  4.84it/s]


In [13]:
len(titleId_list)

619

In [14]:
titleId_list

[758037,
 648419,
 183559,
 774866,
 733074,
 783052,
 728750,
 602910,
 654774,
 796867,
 795297,
 813552,
 806189,
 796075,
 801035,
 759940,
 768536,
 805907,
 761461,
 752414,
 772725,
 21815,
 791256,
 783050,
 807775,
 802039,
 730174,
 741891,
 769663,
 814541,
 812403,
 739115,
 801710,
 803518,
 780414,
 687915,
 814116,
 808711,
 808776,
 800333,
 800034,
 679568,
 805437,
 791629,
 794421,
 756140,
 774302,
 798277,
 801517,
 801827,
 804782,
 800618,
 791062,
 794156,
 772734,
 745654,
 804469,
 734574,
 793944,
 798293,
 802854,
 802551,
 807397,
 808760,
 785749,
 802033,
 797253,
 796368,
 807159,
 776256,
 805702,
 793067,
 811129,
 792317,
 808074,
 791205,
 740132,
 764480,
 807600,
 799868,
 764623,
 772243,
 786979,
 803123,
 814362,
 814594,
 812354,
 811721,
 736277,
 710751,
 758150,
 774044,
 779632,
 795262,
 703844,
 774831,
 803909,
 805193,
 813564,
 759567,
 798664,
 743838,
 799805,
 792694,
 799267,
 811219,
 809706,
 728128,
 813410,
 805691,
 800778,
 7

## 각 웹툰의 정보 크롤링

In [11]:
import requests
import pandas as pd

### 테스트

In [19]:
# 하나의 웹툰에 저장된 정보가 담긴 url api
url2 = "https://comic.naver.com/api/article/list/info?titleId=796075"

head2 = {'User-Agent':
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'}

In [20]:
r2 = requests.get(url2, headers=head2)
# r2.text

In [68]:
r2.json().keys()

dict_keys(['titleId', 'thumbnailUrl', 'posterThumbnailUrl', 'sharedThumbnailUrl', 'titleName', 'webtoonLevelCode', 'author', 'thumbnailBadgeList', 'age', 'publishDescription', 'synopsis', 'favoriteCount', 'favorite', 'firstArticle', 'publishDayOfWeekList', 'rest', 'finished', 'dailyPass', 'curationTagList', 'adBannerList', 'chargeBestChallenge', 'contentsNo', 'new'])

In [17]:
## 모을 데이터 컬럼:
### "titleName, author, age , favoriteCount, publishDayOfWeekList, curationTagList"

#### doc string 활용 내용

# titleName 웹툰 제목
# author 작가
# age 시청 연령
# publishDayOfWeekList 연재 요일
# favoriteCount 관심도 수
# curationTagList 웹툰 내용 관련 해시태그

In [69]:
r2.json()['curationTagList']

[{'id': 796075,
  'tagName': '무협/사극',
  'urlPath': '/webtoon?tab=genre&genre=HISTORICAL',
  'curationType': 'GENRE_HISTORICAL'},
 {'id': 184,
  'tagName': '환골탈태',
  'urlPath': '/curation/list?type=CUSTOM_TAG&id=184',
  'curationType': 'CUSTOM_TAG'},
 {'id': 796075,
  'tagName': '소설원작',
  'urlPath': '/curation/list?type=NOVEL_ORIGIN',
  'curationType': 'NOVEL_ORIGIN'}]

In [19]:
### 만약 분석을 추가한다면?
#### 관심도 수 와 웹툰 랭킹을 연관지을 수 있으려나?? -> 조회순으로 보여지는 각 요일의 순서 == 랭킹이라고 가정.


In [70]:
r2.json()['titleName']

'절대검감'

In [71]:
r2.json()['author']['writers'][0]['name']

'김두루미'

In [72]:
r2.json()['age']['type']

'RATE_12'

In [73]:
r2.json()['favoriteCount']

115387

In [74]:
r2.json()['publishDayOfWeekList'][0]

'MONDAY'

In [21]:
a = []
for i in range(r2.json()['curationTagList'].__len__()):
    a.append(r2.json()['curationTagList'][i]['tagName'])



In [22]:
a
## 해당 리스트를 데이터프레임에서 한 컬럼안에 넣기 위해선 문자열로 바꿔야한다.

['무협/사극', '환골탈태', '소설원작']

In [77]:
## 이후에 잘 추출하기 위해 공백을 없앰.
', '.join(a).replace(" ","")

'무협/사극,환골탈태,소설원작'

In [78]:
b = [
    r2.json()['titleName'],
    r2.json()['author']['writers'][0]['name'],
    r2.json()['age']['type'],
    r2.json()['favoriteCount'],
    r2.json()['publishDayOfWeekList'][0],
    ', '.join(a).replace(" ","")
]
b

['절대검감', '김두루미', 'RATE_12', 115387, 'MONDAY', '무협/사극,환골탈태,소설원작']

In [53]:
c = {
    'Title': [r2.json()['titleName']],
    'Author': [r2.json()['author']['writers'][0]['name']],
    'Age': [r2.json()['age']['type']],
    'Favorite': [r2.json()['favoriteCount']],
    'PublicDay': r2.json()['publishDayOfWeekList'],
    'HashTag': [', '.join(a).replace(" ","")]
}
c

{'Title': ['절대검감'],
 'Author': ['김두루미'],
 'Age': ['RATE_12'],
 'Favorite': [115482],
 'PublicDay': ['MONDAY'],
 'HashTag': ['무협/사극,환골탈태,소설원작']}

In [55]:
m_df=pd.DataFrame(c)

In [57]:
m_df

,Title,Author,Age,Favorite,PublicDay,HashTag
0,절대검감,김두루미,RATE_12,115482,MONDAY,"무협/사극,환골탈태,소설원작"


In [ ]:
## 재귀함수로 데이터프레임을 병합해야함. 아마도? 이건 좀 생각해보자

# 전체 title_id를 통해 데이터프레임 만들기 

In [1]:
import requests
import pandas as pd

In [28]:
# 하나의 웹툰에 저장된 정보가 담긴 url api
### Id 부분은 format으로 동적 변경
test_url = "https://comic.naver.com/api/article/list/info?titleId=774866"

head2 = {'User-Agent':
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'}

In [32]:
t_r= requests.get(test_url, headers=head2)

In [37]:
t_r.json()['curationTagList'].__len__()

3

In [67]:
# 'curationTagList' 부분은 딕셔너리 안에 딕셔너리 구조로 되어 있기 때문에
## 이를 각각의 데이터를 뽑아 리스트 안에 넣었고,
### 한 컬럼 안에 해당 데이터들을 모두 넣기 위해
#### ", ".join() 을 통해 문자열로 변환함.
##### 또한 replace() 를 통해 문자열 간의 공백을 없애고, ',' 로만 구분짓게 함.

tmp = []
for i in range(t_r.json()['curationTagList'].__len__()):
    tmp.append(t_r.json()['curationTagList'][i]['tagName'])
    
", ".join(tmp).replace(" ","")

'무협/사극,사이다,힘숨찐,판무,먼치킨,소설원작'

In [68]:
from tqdm import tqdm

In [69]:
test_url = "https://comic.naver.com/api/article/list/info?titleId={}"

head = {'User-Agent':
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'}

# m_df = pd.DataFrame()
data_list = []

for x in tqdm(titleId_list) :
    t_r= requests.get(test_url.format(x), headers=head)

    # 'curationTagList' 부분은 딕셔너리 안에 딕셔너리 구조로 되어 있기 때문에
    ## 이를 각각의 데이터를 뽑아 리스트 안에 넣었고,
    ### 한 컬럼 안에 해당 데이터들을 모두 넣기 위해
    #### ", ".join() 을 통해 문자열로 변환함.
    ##### 또한 replace() 를 통해 문자열 간의 공백을 없애고, ',' 로만 구분짓게 함.
    tmp = []
    for i in range(t_r.json()['curationTagList'].__len__()):
        tmp.append(t_r.json()['curationTagList'][i]['tagName'])
        
    tags = ", ".join(tmp).replace(" ","")

    temp = {
        'Title': t_r.json()['titleName'],
        'Author': t_r.json()['author']['writers'][0]['name'],
        'Age': t_r.json()['age']['type'],
        'Favorite': t_r.json()['favoriteCount'],
        'PublicDay': t_r.json()['publishDayOfWeekList'][0],
        'HashTag': tags
    }
    data_list.append(temp)




100%|██████████| 619/619 [00:33<00:00, 18.71it/s]


In [73]:
pd.DataFrame(data_list).to_csv("./webtoon.csv", encoding="utf-8-sig")

#### (사용X) `selenium` 을 활용한 크롤링

In [ ]:
# pip install selenium

In [ ]:
from selenium import webdriver
import selenium

In [ ]:
driver = webdriver.Chrome()

In [ ]:
driver.get("https://comic.naver.com/webtoon")

In [ ]:
from selenium.webdriver.common.by import By
driver.find_element(By.CSS_SELECTOR,
                    "#container > div.component_wrap.type2 > div.WeekdayMainView__daily_all_wrap--UvRFc > div:nth-child(1) > ul > li:nth-child(1) > div > a > span").click()

In [ ]:
driver.find_element(By.CSS_SELECTOR,
).click()